In [250]:
#workaround for running jupyter from wsl
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

In [251]:
from dotenv import load_dotenv

load_dotenv()

True

In [252]:
uri = "http://ramp.uni-mannheim.de"
graph_endpoint = os.getenv("GRAPH_ENDPOINT")
username = os.getenv("GRAPH_USERNAME")
password = os.getenv("GRAPH_PASSWORD")

In [253]:
# Initialize an RDFLib Graph
from rdflib import Graph, Namespace

RAMP = Namespace("http://ramp.uni-mannheim.de/")

graph = Graph()

graph.bind("ramp", RAMP)

print(os.getcwd())

# Load ontology from the specified file
graph.parse("./ontology/ontology.ttl", format="turtle")

/home/yannik/coding-native/extractor/sparql


<Graph identifier=N17b5e423aa764a3bb646fad520619470 (<class 'rdflib.graph.Graph'>)>

In [254]:
from scripts import parse_module_catalog, parse_overview


overview = parse_overview("../output/20241112_000328/catalog_overview.json")

split = parse_module_catalog("../output/20241112_000328/split_2_modules_list.json")

In [255]:
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF
import requests

XSD = Namespace("http://www.w3.org/2001/XMLSchema#")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")

# Iterate through each study area and module to create triples
for study_area in overview.studyArea:
    for module in study_area.modules:
        # Create the URI for the module
        module_uri = URIRef(f"{uri}/module/{module.id}")

        # Add RDF type triple
        graph.add((module_uri, RDF.type, RAMP.Module))
        
        # Add literal properties for name and id
        graph.add((module_uri, RAMP.name, Literal(module.name)))
        graph.add((module_uri, RAMP.id, Literal(module.id)))

rdf_data = graph.serialize(format="nt")

# graph.update(f"INSERT DATA {{ {graph.serialize(format='nt')} }}")



In [256]:
from rdflib import BNode


for module in split.modules:
    module_uri = URIRef(f"{uri}/module/{module.id}")

    if module.ects is not None:
        graph.add((module_uri, RAMP.ects, Literal(module.ects, datatype=XSD.int)))
    if module.workloadInPerson is not None:
        graph.add((module_uri, RAMP.workloadInPerson, Literal(module.workloadInPerson, datatype=XSD.int)))
    if module.workloadSelfStudy is not None:
        graph.add((module_uri, RAMP.workloadSelfStudy, Literal(module.workloadSelfStudy, datatype=XSD.int)))
    if module.assessmentForms:
        for i, form in enumerate(module.assessmentForms):
            blank_node = BNode()
            graph.add((module_uri, RAMP.hasAssesment, blank_node))
            graph.add((blank_node, RAMP.hasAssessmentForm, Literal(form.value)))
            if module.examinationDistribution:
                if(len(module.examinationDistribution.split("/")) > i):
                    dist = module.examinationDistribution.split("/")[i] 
                    graph.add((blank_node, RAMP.examinationDistribution, Literal(dist, datatype=XSD.int)))

    if module.examinationDuration is not None:
        graph.add((module_uri, RAMP.examinationDuration, Literal(module.examinationDuration, datatype=XSD.int)))
    if module.additionalPrerequisite:
        graph.add((module_uri, RAMP.additionalPrerequisite, Literal(module.additionalPrerequisite)))
    # Add lists of literals
    for semester in module.offeredIn:
        graph.add((module_uri, RAMP.offeredIn, Literal(semester.value)))
    for recommendedSemester in module.recommendedSemester:
        graph.add((module_uri, RAMP.recommendedSemester, Literal(recommendedSemester, datatype=XSD.int)))
    for literature in module.recommendedLiterature:
        graph.add((module_uri, RAMP.recommendedLiterature, Literal(literature)))
    for prerequisite in module.prerequisiteModules:
        graph.add((module_uri, RAMP.hasPrerequisite, URIRef(f"{uri}/module/{prerequisite}")))
    for further in module.furtherModules:
        graph.add((module_uri, RAMP.hasFurtherModue, Literal(further)))

     # Add lecturer and person in charge as URI references
    for lecturer in module.hasLecturer:
        lecturer_uri = URIRef(f"{uri}/person/{lecturer.name.replace(' ', '_')}")
        graph.add((module_uri, RAMP.hasLecturer, lecturer_uri))
        graph.add((lecturer_uri, RDF.type, FOAF.Person))
    for person_in_charge in module.hasPersonInCharge:
        person_in_charge_uri = URIRef(f"{uri}/person/{person_in_charge.name.replace(' ', '_')}")
        graph.add((module_uri, RAMP.hasPersonInCharge, person_in_charge_uri))
        graph.add((person_in_charge_uri, RDF.type, FOAF.Person))
    


In [257]:
# Serialize and print the graph in Turtle format (optional for debugging)
print(graph.serialize(format="turtle"))

# store ontology in file
graph.serialize(destination="../ontology/ontology-full.ttl", format="turtle")

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix ramp: <http://ramp.uni-mannheim.de/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://ramp.uni-mannheim.de> a owl:Ontology .

ramp:additionalPrerequisite a owl:DatatypeProperty ;
    rdfs:domain ramp:Module .

ramp:ects a owl:DatatypeProperty ;
    rdfs:label "ECTS" ;
    rdfs:domain ramp:Module ;
    rdfs:range xsd:decimal .

ramp:examinationDistribution a owl:AnnotationProperty ;
    rdfs:domain ramp:Module ;
    rdfs:range ramp:AssessmentForm .

ramp:examinationDuration a owl:AnnotationProperty ;
    rdfs:domain ramp:Module ;
    rdfs:range ramp:AssessmentForm .

ramp:hasDegree a owl:ObjectProperty ;
    rdfs:domain ramp:StudyProgram ;
    rdfs:range ramp:Degree .

ramp:hasExaminationForm a owl:ObjectProperty .

ramp:hasLecturer a owl:ObjectProperty ;
    

<Graph identifier=N17b5e423aa764a3bb646fad520619470 (<class 'rdflib.graph.Graph'>)>

In [258]:
from blazegraph_client import create_blazegraph_namespace, delete_blazegraph_namespace

#reset graph

delete_blazegraph_namespace("ramp")

create_blazegraph_namespace("ramp")




Namespace 'ramp' deleted successfully.
Namespace 'ramp' created successfully.


True

In [259]:
# Send the update request to the SPARQL endpoint
# Prepare SPARQL INSERT statement
sparql_update = f"""
INSERT DATA {{
    {graph.serialize(format="turtle")}
}}
"""

# Send the request with Basic Authentication
headers = {
    "Content-Type": "application/sparql-update"
}

response = requests.post(
    url=graph_endpoint,
    data=sparql_update,
    headers=headers,
    auth=HTTPBasicAuth(username, password)
)

if response.status_code == 200:
        print("Data successfully inserted.")
else:
    print("Failed to insert data:", response.status_code, response.text)

Data successfully inserted.
